## Installing Web3 and JSonschema
The code in this cell will install the web3 and jsonschema packages. Web3 is a library for interacting with Ethereum, and jsonschema is a library for validating JSON data. Both of these packages are necessary for the code in the following cells to run properly if working on Google Colab.

In [1]:
#!pip install web3
#!pip install --force-reinstall jsonschema==3.2.0

## Importing Libraries
The code in this cell will import the necessary libraries for the code in the following cells to run properly. The libraries imported are web3, datetime, pandas, requests, and time. Web3 is a library for interacting with Ethereum, datetime is a library for working with dates and times, pandas is a library for data analysis, requests is a library for making HTTP requests, and time is a library for working with time.

In [2]:
from web3 import Web3
from datetime import datetime
import pandas as pd
import requests
import time

## Connecting to Ethereum Network
The code in this cell will connect to the Ethereum network using the Infura API. It will create a web3 object using the Infura URL provided, and then use the `web3.isConnected()` method to check if the connection was successful.

In [3]:
infura_url= 'https://mainnet.infura.io/v3/[YOUR INFURA KEY]'
web3= Web3(Web3.HTTPProvider (infura_url))
web3.isConnected()

True

**Exercise 1**

#### Calculating Sovryn Total Value Locked
The code in this cell will use the requests library to make a GET request to the Sovryn backend API. It will then parse the response and extract the total value locked (TVL) for the Sovryn lending service, automated market makers, protocol, and subprotocols. Finally, it will add all of these values together to calculate the total value locked for the Sovryn protocol.

In [4]:
url="https://backend.sovryn.app/tvl"
response = requests.get(url)
data= response.json()
tvl_lending=data["tvlLending"]["totalUsd"] #lending service
tvl_amm=data["tvlAmm"]["totalUsd"] #automated market makers
tvl_protocol= data["tvlProtocol"]["totalUsd"]
tvl_sub=data["tvlSubprotocols"]["totalUsd"]
sovryn_tvl= tvl_lending + tvl_amm + tvl_protocol + tvl_sub
sovryn_tvl

18317494.722492155

## Babel Fish

According to their webpage BabelFish "absorbs, aggregates and distributes USD-pegged stablecoins", so we are multiplying the balances by USDT price that we query from CoinGecko 

In [8]:
url="https://api.covalenthq.com/v1/30/address/0x1440d19436bEeaF8517896bffB957a88EC95a00F/balances_v2/?quote-currency=USD&format=JSON&nft=false&no-nft-fetch=false&key=ckey_0784e6579a4747b5b627b9d28aa"
response = requests.get(url)
data= response.json()

url="https://api.coingecko.com/api/v3/simple/price?vs_currencies=usd&include_market_cap=true&ids=tether"
response = requests.get(url)  
data_price= response.json()
gecko_price= data_price ['tether']['usd']

The code in the following cells will loop through the data from the CovalentHQ API call and store the balance of each contract in the 'balance' list. It will then calculate the total value locked by summing the balances in the 'balance' list and multiplying them by 10 to the power of the decimals (18) and then multiplying it with the market price from the CoinGecko API call. It will then sum all the balances and store the result in the 'babel_tvl' variable.

In [9]:
decimals=18
balance=[]

for contract in range(len (data ['data']['items'])):
  if int(data ['data']['items'][contract]['balance'])!=0:
    balance.append (int(data ['data']['items'][contract]['balance']))


In [10]:
babel_tvl=0
for item in balance:
  balance_usdt = (item*10**-decimals)*gecko_price
  babel_tvl += balance_usdt

babel_tvl

7848490.254317084

**Exercise 2**

#### Calculating Transaction Counts
The code in this cell will use the web3 library to get the latest block number and then loop through the last 50 blocks. It will store the block numbers in the 'blocks_id' list and the number of transactions in each block in the 'transaction_count' list.

In [16]:
blocks_id=[]
transaction_count=[]
latest = web3.eth.blockNumber
for i in range(0, 50):
  block= (web3.eth.getBlock(latest - i))
  blocks_id.append (block.number)
  transaction_count.append (len(block.transactions))

#### Filtering Transactions
The code in this cell will loop through the blocks and transactions stored in the 'blocks_id' and 'transaction_count' lists. It will then filter out transactions with a value greater than 50 Ether and store the to and from addresses, transaction hash and value in the 'adresses_to', 'address_from', 'transaction_hash' and 'value' lists respectively.

The code in the following cell will create a dataframe from the lists created in the previous cell and will then sort the dataframe by the 'value' column in descending order in order to display the top 10 rows.

In [17]:
adresses_to = []
address_from=[]
transaction_hash =[]
value =[]

for block, count in zip(blocks_id, transaction_count):
  for tx in range (count):
    data= (web3.eth.getTransactionByBlock (block,tx))
    if (web3.fromWei((data.value),'ether')) > 50:
        adresses_to.append (data.to)
        address_from.append (data['from'])
        transaction_hash.append (web3.toHex(data.hash))
        value.append (web3.fromWei((data.value),'ether'))

In [18]:
df = {'value':value, 'transaction_hash':transaction_hash,'adress_from':address_from ,'adress_to':adresses_to}
df = pd.DataFrame(df)
df=df.sort_values(by=['value'], ascending=False)
df.head(10)

,value,transaction_hash,adress_from,adress_to
2,3489.1,0xa9ce075230da1cf849b44f2f30db685bfd912703beb0...,0xd24400ae8BfEBb18cA49Be86258a3C749cf46853,0x479Bc00624e58398f4Cf59D78884d12Fb515790A
7,1547,0xd3ae8fe70a80f0c7ccc70bc0cd5d266954aa3f5aa351...,0x00514112B64626360445D45dae83327c989FacE2,0x7286fCB1f0B9652063325f9d9Dc6fef092D6E711
3,420.64386656,0xd44d7f13c53ce12595d397eab2d2fa9cba6c444b2dd6...,0x9696f59E4d72E237BE84fFD425DCaD154Bf96976,0x2Ca212aF7E65631d46A1cF406427f257D2976A71
6,268.08377833,0x3c9c46894d3f0213203da7265a56a4da9d0e9d7b5092...,0x9696f59E4d72E237BE84fFD425DCaD154Bf96976,0xb5964c355EdF06840E5917A9c9f3d51D2a233B36
8,234,0xb3f47a4e1111147a4ff4f0964ab3c90b8478d1e92968...,0x6277a1073DFBE0Ea087E0eF8741DCE084831E782,0x794958f3Fb8af2e795308fc0948c4F6857D0AfBc
4,153.9965,0x8254e9aa59d438e306b53c4ea6615db18a55d92472e8...,0x267be1C1D684F78cb4F6a176C4911b741E4Ffdc0,0x32d4703e5834F1b474B17DFdB0aC32Cc22575145
1,79.990607357653012784,0xc421f6876f8239587a0f860d7f8cd6c700c2794642e5...,0x285D153589c8fe208C9F00cc966D7A1d32A9A655,0x28C6c06298d514Db089934071355E5743bf21d60
0,70.5,0x84fc5c2542174784c34f62b8b834894a8642af504032...,0x6cFdad0772EDC2EA15076A9706ec7c1F0a791aC1,0x74312363e45DCaBA76c59ec49a7Aa8A65a67EeD3
5,70,0xed9f31e205e9c5f05ca4f82f8052d221400786bb1cf1...,0x1e218a3795479F1A9F0156F6E136db6f1613F7d8,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
9,68.692252406084034466,0x74ee0eccc3049dc2099290fa89c0e71e1258803bcf3d...,0x0c8056914D4e49419EdB73516e18E0B8722593Eb,0xb2dFF401DD22916162C211eba048B2e939344AAd


**Exercise 3**

Go to "solutions-1-kaggle"